In [ ]:
# Step 1: Setup Google Colab Environment
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# Step 2: Install Required Libraries
!apt-get install unrar
!pip install rarfile


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Step 3: Download and Prepare the Data
import gdown
import rarfile
import os
# Define URLs and file names
labeled_data_url = 'https://drive.google.com/uc?id=1AWCbC8LOAGo5Mltyd1gmlPr8vlk5fXl9'
random_data_url = 'https://drive.google.com/uc?id=1yS1vPl0Uu2EMocKbwLI3fiNWhwjhDh0Y'
labeled_data_rar = 'labeled_data.rar'
random_data_rar = 'random_data.rar'

# Download the .rar files
gdown.download(labeled_data_url, labeled_data_rar, quiet=False)
gdown.download(random_data_url, random_data_rar, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1AWCbC8LOAGo5Mltyd1gmlPr8vlk5fXl9
From (redirected): https://drive.google.com/uc?id=1AWCbC8LOAGo5Mltyd1gmlPr8vlk5fXl9&confirm=t&uuid=7d3ccdef-3080-491a-b24f-11a77699dc44
To: /content/labeled_data.rar
100%|██████████| 167M/167M [00:12<00:00, 13.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1yS1vPl0Uu2EMocKbwLI3fiNWhwjhDh0Y
From (redirected): https://drive.google.com/uc?id=1yS1vPl0Uu2EMocKbwLI3fiNWhwjhDh0Y&confirm=t&uuid=31683a26-3ecc-43a6-8c81-c4cb5803c8fc
To: /content/random_data.rar
100%|██████████| 155M/155M [00:14<00:00, 10.9MB/s]


'random_data.rar'

In [ ]:


# Define a function to extract .rar files
def extract_rar(rar_path, extract_to):
    try:
        with rarfile.RarFile(rar_path) as rf:
            rf.extractall(extract_to)
        print(f"Extracted {rar_path} to {extract_to}")
    except rarfile.BadRarFile:
        print(f"Error: {rar_path} is not a valid rar file or it is corrupted.")

# Create directories if they don't exist
os.makedirs('labeled_data', exist_ok=True)
os.makedirs('random_data', exist_ok=True)

# Extract the .rar files
extract_rar(labeled_data_rar, 'labeled_data')
extract_rar(random_data_rar, 'random_data')


Extracted labeled_data.rar to labeled_data
Extracted random_data.rar to random_data


In [ ]:

# Proceed with loading and preprocessing the data
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Define paths
train_dir = 'labeled_data/TIRES'
valid_dir = 'labeled_data/TIRES'
random_data_dir = 'random_data/random'

# Image data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 2402 images belonging to 4 classes.
Found 598 images belonging to 4 classes.


In [ ]:

# Build and Train the Model
model = Sequential([
    VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=256,
    validation_data=validation_generator
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/256


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


76/76 ━━━━━━━━━━━━━━━━━━━━ 77s 625ms/step - accuracy: 0.3784 - loss: 2.8088 - val_accuracy: 0.4097 - val_loss: 1.3447
Epoch 2/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 31s 268ms/step - accuracy: 0.4858 - loss: 1.1822 - val_accuracy: 0.8746 - val_loss: 0.5055
Epoch 3/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 41s 270ms/step - accuracy: 0.9099 - loss: 0.2997 - val_accuracy: 0.8662 - val_loss: 0.3807
Epoch 4/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 22s 281ms/step - accuracy: 0.9363 - loss: 0.2161 - val_accuracy: 0.9164 - val_loss: 0.2968
Epoch 5/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 41s 284ms/step - accuracy: 0.9663 - loss: 0.1261 - val_accuracy: 0.9465 - val_loss: 0.2151
Epoch 6/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 43s 310ms/step - accuracy: 0.9254 - loss: 0.2512 - val_accuracy: 0.8696 - val_loss: 0.4023
Epoch 7/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 23s 287ms/step - accuracy: 0.8917 - loss: 0.3202 - val_accuracy: 0.9231 - val_loss: 0.2255
Epoch 8/256
76/76 ━━━━━━━━━━━━━━━━━━━━ 40s 273ms/step - accuracy: 0.9582 - loss: 0.1175 - val_accuracy

In [ ]:
import pandas as pd
# Evaluate the Model
results = []

# Load random images
for img_name in os.listdir(random_data_dir):
    img_path = os.path.join(random_data_dir, img_name)

    # Preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predict the class
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_index]

    # Print the result (optional)
    print(f'{img_name}: {class_label}')

    # Append result to the list
    results.append({
        'Image Name': img_name,
        'Predicted Label': class_label
    })

# Convert the results into a DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df.to_excel('classification_results.xlsx', index=False)

print("Results saved to classification_results.xlsx")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
146118389423.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
14948753508.JPG: OCR
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
151008274452.JPG: OCR
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1539956577772.JPG: front
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
148753992347.JPG: front
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
146074421994.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
147947988438.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
145513373097.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1540600572739.JPG: front
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
144052802229.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
146499128210.JPG: front
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
143438958698.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
144243014257.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
142895208641.JPG: side
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
151620508352.JPG: MultiTires
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1539298288934.JPG: front
1/1 ━━━━━━━━━